# Preliminary Notebook

## 1. Import and Install Dependencies

In [7]:
%pip install tensorflow-macos opencv-python mediapipe-silicon sklearn matplotlib
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2 # opencv
import numpy as np
import os # easier file path handling
from matplotlib import pyplot as plt # im.show for easy visualization
import time # to insert "sleep" in between frames
import mediapipe as mp # for accessing and reading from webcam

## 6. Preprocess Data and Create Labels and Features

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
# path for exported data (numpy arrays)
#DATA_PATH = os.path.join('MP_Data')
DATA_PATH = os.path.join('MP_Data_test')

# actions to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])
#actions = np.array ( ['alligator', 'radio', 'moon', 'sleep', 'grandpa', 'tiger', 'pencil', 'sleepy', 'grandma', 'chocolate'])

# 30 videos of data
no_sequences = 30

# each video with 30 frames
sequence_length = 30

In [11]:
# create label map (dict, our .json file)
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
sequences, labels = [], [] # sequences will be x data, labels will be y data
# loop over all actions (words)
for action in actions: 
    # loop over all sequences (videos)
    for sequence in range(no_sequences): 
        window = [] # represents all frames of particular sequence (video)
        # loop through each frame
        for frame_num in range(sequence_length): 
            # load up current frame (frame_num)
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res) # append to one video
        sequences.append(window) # append all videos to sequence (for a word)
        labels.append(label_map[action])

## Feeding in preprocessed kaggle data

In [13]:
#loading our preprocessed kaggle data
X = np.load('/Users/ronjaweiblen/Bootcamp/Capstone_Project_SignMeUp/data/X-data.npy')
y = np.load('/Users/ronjaweiblen/Bootcamp/Capstone_Project_SignMeUp/data/y-data.npy')

actions = np.array ( ['alligator', 'radio', 'moon', 'sleep', 'grandpa', 'tiger', 'pencil', 'sleepy', 'grandma', 'chocolate'])


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Feeding in our sample data

In [15]:
#loading our own sample data
X_j = np.array(sequences) # shape: (90, 30, 1662)
y_j = to_categorical(labels).astype(int) # one-hot encoded labels (words)

In [16]:
X_j_train, X_j_test, y_j_train, y_j_test = train_test_split(X_j, y_j, test_size=0.05)

In [17]:
X_j_train.shape, X_train.shape

((85, 30, 1662), (315, 30, 1086))

In [18]:
y_j_train.shape, y_train.shape

((85, 3), (315, 10))

**Problem: Number of signs changes the shape. Where do we change this in the model?**

## 7. Build and Train LSTM Neural Network

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [21]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1086))) #changed to 1086 columns
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [22]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# categorical_crossentropy must be used for multiclass classification model! 

In [23]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
# advantage of using mediapipe holistic model is you don't need additional data generator to build up a pipeline of data. Training data fits all into memory.

Epoch 1/2000


2023-04-06 15:56:42.847866: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


10/10 [==============================] - 3s 46ms/step - loss: 2.3310 - categorical_accuracy: 0.0794
Epoch 2/2000
10/10 [==============================] - 0s 48ms/step - loss: 2.2956 - categorical_accuracy: 0.0921
Epoch 3/2000
10/10 [==============================] - 0s 46ms/step - loss: 2.2911 - categorical_accuracy: 0.1175
Epoch 4/2000
10/10 [==============================] - 0s 42ms/step - loss: 2.2977 - categorical_accuracy: 0.1175
Epoch 5/2000
10/10 [==============================] - 0s 40ms/step - loss: 2.2973 - categorical_accuracy: 0.1365
Epoch 6/2000
10/10 [==============================] - 0s 40ms/step - loss: 2.2952 - categorical_accuracy: 0.1270
Epoch 7/2000
10/10 [==============================] - 0s 40ms/step - loss: 2.2892 - categorical_accuracy: 0.1397
Epoch 8/2000
10/10 [==============================] - 0s 39ms/step - loss: 2.2753 - categorical_accuracy: 0.1175
Epoch 9/2000
10/10 [==============================] - 0s 39ms/step - loss: 2.2715 - categorical_accuracy: 0.1

In [24]:
# %load_ext tensorboard

In [25]:
# to run it in the notebook: 
# %tensorboard --logdir=./Logs/train --port=8008

# to run it in terminal: python3 -m tensorboard --logdir=./Logs/train --port=8008
# then copy+paste this into your internet browser: localhost:8008

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            294656    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 449,450
Trainable params: 449,450
Non-trai

## Make Predictions

In [27]:
res = model.predict(X_test)

1/1 [==============================] - 1s 551ms/step


Some single example predictions :D

In [28]:
actions[np.argmax(res[0])] # predictions

'alligator'

In [29]:
actions[np.argmax(y_test[0])] # actual

'chocolate'

## 9. Save Weights

In [31]:
model.save('Easter_model_10signs.h5')